# 🔥 Heatmap - Accessibilité aux médecins (France entière)

Visualisation rapide et légère avec heatmap (sans info-bulles)

## 📦 Imports

In [1]:
import geopandas as gpd
import pandas as pd
import folium
from folium.plugins import HeatMap
import matplotlib.pyplot as plt
from pathlib import Path

print("✅ Imports OK")

✅ Imports OK


## 📥 Chargement des données

In [2]:
# Charger IRIS France entière
print("📥 Chargement des IRIS...")
gdf_iris = gpd.read_file("../data/geo/iris_france.gpkg")

print(f"✅ {len(gdf_iris):,} IRIS chargés")

📥 Chargement des IRIS...
✅ 48,512 IRIS chargés


In [3]:
file_name = "apl_medecins_generalistes.xlsx"
# PATHS
data_path = Path("/Users/jean-jacques/code/jjchabutDataCRM/sante-territoires/data/raw/offres_soins")

full_path = data_path / file_name

xl = pd.ExcelFile(full_path)
print(f'Sheet du fichier : {xl.sheet_names}')

# 1. Charger l'onglet APL 2023 
# On saute les 8 premières lignes pour arriver aux noms de colonnes (Code commune, etc.)
df_2023 = pd.read_excel(
    full_path, 
    sheet_name='APL 2023', 
    skiprows=8,
    dtype={'Code commune': str} # Important pour garder les "0" au début des codes postaux/Insee
)

# 2. Charger l'onglet Paramètres
# Les données de pondération commencent à la ligne 21
df_params = pd.read_excel(
    full_path, 
    sheet_name='Paramètres', 
    skiprows=21
)


Sheet du fichier : ['Paramètres', 'APL 2022', 'APL 2023']


## 🔗 Jointure IRIS + APL

In [4]:
# Jointure
gdf_final = gdf_iris.merge(
    df_2023,
    left_on='code_insee',
    right_on='Code commune INSEE',
    how='inner'  # Garder seulement les IRIS avec APL
)

print(f"✅ {len(gdf_final):,} IRIS avec APL")

✅ 48,492 IRIS avec APL


## 📍 Extraction des coordonnées

In [5]:
# Reprojeter en WGS84 pour obtenir lat/lon
print("🌍 Reprojection en WGS84...")
gdf_final_wgs84 = gdf_final.to_crs(epsg=4326)

# Extraire les coordonnées du centroïde de chaque IRIS
gdf_final_wgs84['longitude'] = gdf_final_wgs84.geometry.centroid.x
gdf_final_wgs84['latitude'] = gdf_final_wgs84.geometry.centroid.y

print("✅ Coordonnées extraites")
print("\n🔍 Aperçu :")
print(gdf_final_wgs84[['nom_commune', 'latitude', 'longitude']].head())

🌍 Reprojection en WGS84...
✅ Coordonnées extraites

🔍 Aperçu :
                   nom_commune   latitude  longitude
0                    Bischheim  48.632568   7.782758
1  Marseille 2e Arrondissement  43.324823   5.357664
2                       Quéven  47.769735  -3.442135
3                  Romainville  48.894659   2.433999
4             Marolles-en-Brie  48.748145   2.561351


/var/folders/vr/hhxf111n55b91l_9j5qxjdrm0000gn/T/ipykernel_51931/2147017105.py:6: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_final_wgs84['longitude'] = gdf_final_wgs84.geometry.centroid.x
/var/folders/vr/hhxf111n55b91l_9j5qxjdrm0000gn/T/ipykernel_51931/2147017105.py:7: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_final_wgs84['latitude'] = gdf_final_wgs84.geometry.centroid.y


## 🔍 Identifier la colonne APL

In [6]:
# Trouver la colonne APL
apl_cols = [col for col in gdf_final_wgs84.columns if 'apl' in col.lower() or 'acces' in col.lower()]
print(f"🔍 Colonnes APL trouvées : {apl_cols}")

# ⚠️ ADAPTER si besoin
colonne_apl = apl_cols[0] if apl_cols else 'APL'
print(f"✅ Colonne sélectionnée : {colonne_apl}")

🔍 Colonnes APL trouvées : ['APL aux médecins généralistes', 'APL aux médecins généralistes de 65 ans et moins ', 'APL aux médecins généralistes de 62 ans et moins ', 'APL aux médecins généralistes de 60 ans et moins ']
✅ Colonne sélectionnée : APL aux médecins généralistes


## 🔥 Option 1 : Heatmap simple (déserts médicaux)

In [15]:
# Identifier les zones avec APL faible (déserts médicaux)
seuil_desert = 2.5  # À ajuster

deserts = gdf_final_wgs84[gdf_final_wgs84[colonne_apl] < seuil_desert]

print(f"🏜️ {len(deserts):,} IRIS en désert médical (APL < {seuil_desert})")
print(f"📊 Soit {len(deserts)/len(gdf_final_wgs84)*100:.1f}% des IRIS")

🏜️ 13,968 IRIS en désert médical (APL < 2.5)
📊 Soit 28.8% des IRIS


In [8]:
# Créer la carte
m = folium.Map(
    location=[46.5, 2.5],  # Centre de la France
    zoom_start=6,
    tiles='CartoDB positron'
 )

# Préparer les données pour la heatmap
# Format : [[lat, lon, poids], ...]
heat_data = [
    [row['latitude'], row['longitude'], 1]  # Poids = 1 (intensité uniforme)
    for idx, row in deserts.iterrows()
 ]

# Ajouter la heatmap
HeatMap(
    heat_data,
    radius=15,           # Rayon de diffusion
    blur=20,             # Flou
    max_zoom=13,
    gradient={
        0.0: 'blue',
        0.5: 'yellow',
        1.0: 'red'
    }
 ).add_to(m)

# Titre
title_html = '''
<div style="position: fixed; 
            top: 10px; left: 50px; 
            background-color: white;
            border: 2px solid grey;
            border-radius: 5px;
            z-index: 9999;
            padding: 10px;
            box-shadow: 2px 2px 6px rgba(0,0,0,0.3);">
    <h4 style="margin: 0;">🔥 Déserts médicaux en France</h4>
    <p style="margin: 5px 0 0 0; font-size: 12px;">APL < 2.5</p>
</div>
'''
m.get_root().html.add_child(folium.Element(title_html))

print("✅ Heatmap créée")

# Essayer l'affichage direct Folium
m

✅ Heatmap créée


## 🔥 Option 2 : Heatmap pondérée (intensité variable)

In [9]:
# Carte avec intensité variable selon l'APL
m2 = folium.Map(
    location=[46.5, 2.5],
    zoom_start=6,
    tiles='CartoDB positron'
)

# Inverser l'APL pour que faible APL = forte intensité
max_apl = gdf_final_wgs84[colonne_apl].max()

heat_data_weighted = [
    [
        row['latitude'], 
        row['longitude'], 
        max_apl - row[colonne_apl]  # Poids inversé
    ]
    for idx, row in gdf_final_wgs84.iterrows()
    if pd.notna(row[colonne_apl])
]

# Ajouter la heatmap pondérée
HeatMap(
    heat_data_weighted,
    radius=12,
    blur=15,
    max_zoom=13,
    gradient={
        0.0: 'green',
        0.5: 'yellow',
        1.0: 'red'
    }
).add_to(m2)

# Titre
title2_html = '''
<div style="position: fixed; 
            top: 10px; left: 50px; 
            background-color: white;
            border: 2px solid grey;
            border-radius: 5px;
            z-index: 9999;
            padding: 10px;
            box-shadow: 2px 2px 6px rgba(0,0,0,0.3);">
    <h4 style="margin: 0;">🔥 Accessibilité aux médecins - France</h4>
    <p style="margin: 5px 0 0 0; font-size: 12px;">Rouge = Faible accessibilité</p>
</div>
'''
m2.get_root().html.add_child(folium.Element(title2_html))

print("✅ Heatmap pondérée créée")

# Afficher
m2

✅ Heatmap pondérée créée


## 🔥 Option 3 : Heatmap par région

In [10]:
# Exemple : Heatmap des Hauts-de-France
gdf_hdf = gdf_final_wgs84[gdf_final_wgs84['code_insee'].astype(str).str.startswith(('02', '59', '60', '62', '80'))]

print(f"📍 {len(gdf_hdf):,} IRIS dans les Hauts-de-France")

# Centre sur Lille
m3 = folium.Map(
    location=[50.6, 3.0],
    zoom_start=8,
    tiles='CartoDB positron'
)

# Heatmap région
heat_data_hdf = [
    [row['latitude'], row['longitude'], max_apl - row[colonne_apl]]
    for idx, row in gdf_hdf.iterrows()
    if pd.notna(row[colonne_apl])
]

HeatMap(
    heat_data_hdf,
    radius=10,
    blur=12,
    max_zoom=13,
    gradient={'0.0': 'green', '0.5': 'yellow', '1.0': 'red'}
).add_to(m3)

# Titre
title3_html = '''
<div style="position: fixed; 
            top: 10px; left: 50px; 
            background-color: white;
            border: 2px solid grey;
            border-radius: 5px;
            z-index: 9999;
            padding: 10px;
            box-shadow: 2px 2px 6px rgba(0,0,0,0.3);">
    <h4 style="margin: 0;">🔥 Accessibilité - Hauts-de-France</h4>
</div>
'''
m3.get_root().html.add_child(folium.Element(title3_html))

print("✅ Heatmap Hauts-de-France créée")

m3

📍 5,003 IRIS dans les Hauts-de-France
✅ Heatmap Hauts-de-France créée


## 💾 Sauvegarder les cartes

In [11]:
# Sauvegarder (très rapide !)
m.save('../outputs/figures/heatmap_deserts_medicaux_france.html')
m2.save('../outputs/figures/heatmap_apl_france.html')
m3.save('../outputs/figures/heatmap_hauts_de_france.html')

print("✅ 3 cartes sauvegardées dans outputs/figures/")
print("⚡ Génération ultra-rapide !")

✅ 3 cartes sauvegardées dans outputs/figures/
⚡ Génération ultra-rapide !


## 📊 Stats rapides

In [12]:
# Stats par région (code département)
gdf_final_wgs84['dept'] = gdf_final_wgs84['code_insee'].astype(str).str[:2]

stats_dept = gdf_final_wgs84.groupby('dept')[colonne_apl].agg([
    ('APL moyen', 'mean'),
    ('APL min', 'min'),
    ('APL max', 'max'),
    ('Nb IRIS', 'count')
]).sort_values('APL moyen')

print("📊 Top 10 départements avec APL le plus faible :")
print(stats_dept.head(10))

📊 Top 10 départements avec APL le plus faible :
     APL moyen APL min APL max  Nb IRIS
dept                                   
18    1.931395   0.427   3.561      332
2B    2.027559     0.0  10.298      254
28    2.227772   0.779   4.434      416
77    2.232983   0.146   4.262      762
45    2.234948    0.34   4.805      444
48    2.242038     0.0  10.799      158
01    2.389038     0.0   5.045      447
58    2.390452     0.0   6.402      341
95    2.393241   1.033   4.592      561
07    2.415291     0.0   7.143      361


In [13]:
# Ouvrir la carte dans le navigateur automatiquement
import webbrowser
webbrowser.open('/Users/jean-jacques/code/jjchabutDataCRM/sante-territoires/outputs/figures/heatmap_deserts_medicaux_france.html')

True

In [14]:
# Heatmap Plotly interactive pour les déserts médicaux
import plotly.express as px
import pandas as pd

# Créer un DataFrame à partir des données des déserts
df_deserts = deserts[['latitude', 'longitude']].copy()
df_deserts['poids'] = 1  # Intensité uniforme

# Afficher la heatmap Plotly
fig = px.density_mapbox(
    df_deserts,
    lat='latitude',
    lon='longitude',
    z='poids',
    radius=15,
    center=dict(lat=46.5, lon=2.5),
    zoom=5,
    mapbox_style='carto-positron',
    title='Déserts médicaux en France (APL < 2.5)'
 )
fig.show()

/var/folders/vr/hhxf111n55b91l_9j5qxjdrm0000gn/T/ipykernel_51931/3486743948.py:10: DeprecationWarning: *density_mapbox* is deprecated! Use *density_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/
  fig = px.density_mapbox(
